In [1]:
## Instalaciones

%pip install torch
%pip install open3d
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
## Dependencias

from typing import List
import torch
import os
import open3d as o3d
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from model import PointNetClassifier, PointNetLoss, PointNetKAN
from modelnet10 import ModelNetClass, ModelNet, DatasetType
from utils.csv import save_loss_dict
from utils.transformation import (Normalization,
                                  Rotation, Translation, Reflection, Scale,
                                  DropRandom, DropSphere, Jittering, Noise)
from trainer import PointNetTrainer


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}.")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using cuda.


In [3]:
# parámetros globales
checkpoint_freq = 10

# parámetros del dataset
classes = [label for label in ModelNetClass]
batch_size = 32
dim = 3
num_points = 1024
num_classes = len(classes)

# hiperparámetros
num_global_feats = 1024     # número de features globales calculadas
learning_rate = 0.001
reg_weight = 0.001
gamma = 2                   # Recomendado por el paper de focal loss

# dataset de entrenamiento
t = [Rotation(), Reflection(), Scale(max_ratio=2.0),
    DropRandom(), Jittering(), Noise()]

train_data = ModelNet(classes, DatasetType.TRAIN, repetitions=1, transformations=t, normalize=True)
validation_data = ModelNet(classes, DatasetType.VALIDATION, repetitions=1, transformations=t, normalize=True)
    
# TODO: Más adelante usar alpha para clases imbalanceadas

In [ ]:
# Función de entrenamiento
def train(
        epochs: int,
        name: str,
        num_global_feats: int,
        learning_rate: int,
        use_scheduler: bool,
        alpha: List[int],
        gamma: int,
        reg_weight: int,
        use_kan: bool,
        ignore_Tnet: bool,
):
    if not use_kan:
        classifier = PointNetClassifier(dim, num_points, num_global_feats, num_classes, ignore_Tnet=ignore_Tnet).to(DEVICE)
    else:
        classifier = PointNetKAN(dim, num_points, num_classes, scaling = 3.0).to(DEVICE)
    optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)
    if DEVICE == "cuda" and use_scheduler:
        scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.01, step_size_up=2000, cycle_momentum=False)
    else:
        scheduler = None
    
    trainer = PointNetTrainer(
        name=name,
        model=classifier,
        optimizer=optimizer,
        scheduler=scheduler,
        criterion=PointNetLoss(alpha=alpha, gamma=gamma, reg_weight=reg_weight, size_average=True).to(DEVICE),
        device=DEVICE,
        train_loader=DataLoader(train_data, batch_size=batch_size, shuffle=True),
        val_loader=DataLoader(validation_data, batch_size=batch_size, shuffle=False),
        checkpoint_dir=os.path.join(os.getcwd(), "checkpoint"),
        checkpoint_freq=checkpoint_freq
    )

    loss_dict, best_epoch, best_loss, best_acc = trainer.fit(epochs=epochs)
    #save_loss_dict(loss_dict, os.path.join(os.getcwd(), "csv", f"{name}_loss_dict.csv"))
    print(f"{name} | Best model @ epoch {best_epoch}: loss = {best_loss:.4f}, acc = {best_acc:.4f}")

# Instancias de entrenamiento
EPOCHS=30

In [5]:
train(epochs=EPOCHS, name="base", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=0, reg_weight=reg_weight, use_kan=False, ignore_Tnet=False)

╒══════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch    │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞══════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1  │       2.2025 │      0.3148 │     1.7249 │    0.3992 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2  │       1.8878 │      0.3968 │     1.6577 │    0.4496 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 3  │       1.7254 │      0.443  │     1.4232 │    0.4929 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 4  │       1.6252 │      0.4739 │     1.4895 │    0.464  │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 5  │       1.555  │      0.4935 │     1.6423 │    0.4229 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 6  │       1.5227 │      0.4894 │     1.3635 │    0.5229 │
├──────────┼──────────────┼─────────────┼────────────┼────────

In [6]:
alpha = [3991/106, 3991/515, 3991/889, 3991/200, 3991/200, 3991/465, 3991/200, 3991/680, 3991/392, 3991/344]
train(epochs=EPOCHS, name="alpha", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=alpha, gamma=0, reg_weight=reg_weight, use_kan=False, ignore_Tnet=False)

╒══════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch    │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞══════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1  │      22.5576 │      0.2442 │    22.1489 │    0.2531 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2  │      20.624  │      0.3231 │    18.2661 │    0.385  │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 3  │      19.1012 │      0.3612 │    17.6065 │    0.4181 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 4  │      18.4147 │      0.3856 │    16.8735 │    0.4002 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 5  │      17.7403 │      0.4079 │    16.0113 │    0.4235 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 6  │      17.2958 │      0.4122 │    16.2099 │    0.436  │
├──────────┼──────────────┼─────────────┼────────────┼────────

In [7]:
train(epochs=EPOCHS, name="scheduler", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=True, alpha=None, gamma=0, reg_weight=reg_weight, use_kan=False, ignore_Tnet=False)

╒══════════╤══════════════╤═════════════╤═════════════╤═══════════╕
│ Epoch    │   Train Loss │   Train Acc │    Val Loss │   Val Acc │
╞══════════╪══════════════╪═════════════╪═════════════╪═══════════╡
│ Epoch 1  │       2.3482 │      0.2955 │      1.7918 │    0.4054 │
├──────────┼──────────────┼─────────────┼─────────────┼───────────┤
│ Epoch 2  │       1.9415 │      0.4025 │      1.7214 │    0.4481 │
├──────────┼──────────────┼─────────────┼─────────────┼───────────┤
│ Epoch 3  │       1.8543 │      0.43   │      1.5535 │    0.455  │
├──────────┼──────────────┼─────────────┼─────────────┼───────────┤
│ Epoch 4  │       1.8284 │      0.4325 │      1.9028 │    0.3956 │
├──────────┼──────────────┼─────────────┼─────────────┼───────────┤
│ Epoch 5  │       1.679  │      0.4546 │      1.774  │    0.4656 │
├──────────┼──────────────┼─────────────┼─────────────┼───────────┤
│ Epoch 6  │       1.594  │      0.4715 │      1.4878 │    0.501  │
├──────────┼──────────────┼─────────────┼───────

In [8]:
train(epochs=EPOCHS, name="gamma", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=gamma, reg_weight=reg_weight, use_kan=False, ignore_Tnet=False)

╒══════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch    │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞══════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1  │       1.7602 │      0.3167 │     1.3212 │    0.4073 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2  │       1.4556 │      0.3951 │     1.248  │    0.4358 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 3  │       1.3983 │      0.4223 │     1.3042 │    0.384  │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 4  │       1.3429 │      0.4428 │     1.1243 │    0.4844 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 5  │       1.2495 │      0.4611 │     1.0592 │    0.515  │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 6  │       1.1928 │      0.4747 │     1.0477 │    0.5156 │
├──────────┼──────────────┼─────────────┼────────────┼────────

In [9]:
train(epochs=EPOCHS, name="noreg", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=0, reg_weight=0, use_kan=False, ignore_Tnet=False)

╒══════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch    │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞══════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1  │       1.9657 │      0.34   │     9.0007 │    0.086  │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2  │       1.6864 │      0.4262 │     1.451  │    0.4827 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 3  │       1.5665 │      0.463  │     1.3819 │    0.5248 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 4  │       1.4988 │      0.4788 │     1.3895 │    0.509  │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 5  │       1.4248 │      0.5022 │     1.3493 │    0.5188 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 6  │       1.3971 │      0.5074 │     1.2467 │    0.5571 │
├──────────┼──────────────┼─────────────┼────────────┼────────

In [11]:
train(epochs=EPOCHS, name="kan", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=0, reg_weight=0, use_kan=True, ignore_Tnet=False)

╒══════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch    │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞══════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1  │       3.9876 │      0.2959 │     3.3551 │    0.3652 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2  │       2.9594 │      0.3813 │     3.0981 │    0.345  │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 3  │       2.6084 │      0.41   │     2.0028 │    0.41   │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 4  │       2.4018 │      0.4332 │     2.7382 │    0.4167 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 5  │       2.5936 │      0.4273 │     2.5671 │    0.4146 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 6  │       2.5679 │      0.433  │     3.1454 │    0.4012 │
├──────────┼──────────────┼─────────────┼────────────┼────────

In [12]:
train(epochs=EPOCHS, name="noTnet", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=0, reg_weight=0, use_kan=False, ignore_Tnet=True)

╒══════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch    │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞══════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1  │       1.5422 │      0.4907 │     1.2932 │    0.5535 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2  │       1.2093 │      0.5685 │     1.4354 │    0.5123 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 3  │       1.1283 │      0.5953 │     1.1385 │    0.5967 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 4  │       1.0791 │      0.6127 │     1.1365 │    0.5975 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 5  │       1.0507 │      0.6196 │     1.0844 │    0.6123 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 6  │       1.0298 │      0.6216 │     1.2004 │    0.5671 │
├──────────┼──────────────┼─────────────┼────────────┼────────

In [13]:
train(epochs=EPOCHS, name="global", num_global_feats=512, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=0, reg_weight=reg_weight, use_kan=False, ignore_Tnet=False)

╒══════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch    │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞══════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1  │       2.1965 │      0.3433 │     1.7054 │    0.4188 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2  │       1.8168 │      0.4271 │     1.5694 │    0.4467 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 3  │       1.7144 │      0.4547 │     1.4529 │    0.4823 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 4  │       1.6102 │      0.4741 │     1.4997 │    0.471  │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 5  │       1.5945 │      0.481  │     1.3777 │    0.5146 │
├──────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 6  │       1.5333 │      0.5042 │     1.4092 │    0.5065 │
├──────────┼──────────────┼─────────────┼────────────┼────────

# Dataset Testeo

In [19]:
# dataset de prueba
base_test_data = ModelNet(classes, DatasetType.TEST, repetitions=1, preserve_original=False,
                          transformations=[Normalization()])
affine_test_data = ModelNet(classes, DatasetType.TEST, repetitions=1, preserve_original=False,
                          transformations=[Normalization(),
                                           Rotation(), Reflection(), Scale(max_ratio=2.0),
                                           Normalization()])
complex_test_data = ModelNet(classes, DatasetType.TEST, repetitions=1, preserve_original=False,
                          transformations=[Normalization(),
                                           Rotation(), Reflection(), Scale(max_ratio=2.0),
                                           DropRandom(), Jittering(), Noise(),
                                           Normalization()])

In [ ]:
def test_it(classifier_path: str, num_global_feats=num_global_feats, use_kan=False, ignore_Tnet=False):

    for data_name, data in [["base", base_test_data], ["affine", affine_test_data], ["complex", complex_test_data]]:
        data_loader = DataLoader(data, batch_size=batch_size, shuffle=False)
    
        if not use_kan:
            classifier = PointNetClassifier(dim, num_points, num_global_feats, num_classes, ignore_Tnet=ignore_Tnet).to(DEVICE)
        else:
            classifier = PointNetKAN(dim, num_points, num_classes, scaling = 3.0).to(DEVICE)
        classifier.load_state_dict(torch.load(classifier_path))

        with torch.no_grad():
            classifier = classifier.eval()
            correct = 0
            
            for pcds, labels in data_loader:
                pcds = pcds.to(DEVICE)
                labels = labels.squeeze().to(DEVICE)
                
                # Hacer predicciones
                out, _, _ = classifier(pcds)
            
                # Calculamos las elecciones
                pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
                
                # Elecciones correctas, acumuladas
                correct += pred_choice.eq(labels.data).cpu().sum().item()

            test_acc = correct / float(len(data))
            print(f"\tAccuracy on {data_name} dataset:\t", test_acc)

# Tests
paths = ["base", "alpha", "scheduler", "gamma", "noreg", "kan", "noTnet", "global"]
_dir = os.path.join(os.getcwd(), "checkpoint", "best_model")
for path in paths:
    print(f"'{path.capitalize()}' classifier test results:")
    if path == "kan": test_it(os.path.join(_dir, path+"_best_model.pth"), use_kan=True)
    elif path == "noTnet": test_it(os.path.join(_dir, path+"_best_model.pth"), ignore_Tnet=True)
    elif path == "global": test_it(os.path.join(_dir, path+"_best_model.pth"), num_global_feats=512)
    else: test_it(os.path.join(_dir, path+"_best_model.pth"))

'Base' classifier test results:
'Alpha' classifier test results:
'Scheduler' classifier test results:
'Gamma' classifier test results:
'Noreg' classifier test results:
'Kan' classifier test results:
	Accuracy on base dataset:	 0.8590308370044053
	Accuracy on affine dataset:	 0.16079295154185022
	Accuracy on complex dataset:	 0.1762114537444934
'Notnet' classifier test results:
	Accuracy on base dataset:	 0.8755506607929515
	Accuracy on affine dataset:	 0.3039647577092511
	Accuracy on complex dataset:	 0.3634361233480176
'Global' classifier test results:


KeyboardInterrupt: 